In [2]:
import pandas as pd
from pprint import pprint
import glob
import pandas as pd
import random
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from config import event_types
random.seed(42)
pd.set_option('display.max_rows', None)
# jupyter nbconvert --to script 04b_combine_results_find_disagreement.ipynb
# sudo kill -9 $(nvidia-smi | awk 'NR>8 {print $5}' | grep -E '^[0-9]+$')
version = 2
from config import event_types

def prepare_df(df, type="biolord"):
    global event_types
    df = df.copy()
    print("Loaded file len: ",len(df))
    df.columns = ['Document' if i=='DOCUMENT' else i for i in df.columns]
    df['Sent_ID'] = df['Events'].apply(lambda x: [f"{i:04d}" for i in range(len(x))])
    df = df.explode(["Sent_ID","Events"])
    print("After exploding file len: ",len(df))
    df['UID'] = df['SUBJECT_ID'].astype(str) + "_" + df['ROW_ID'].astype(str) + "_" + df['Sent_ID'].astype(str)
    df = df.dropna(subset="Events")
    df['Event_Name'] = df['Events'].apply(lambda x: x['event'])
    df['Sentence'] = df['Events'].apply(lambda x: x['text'])
    df['Time'] = df['Events'].apply(lambda x: x['event_detection_time'])
    
    if type == "dictionary":
        df['Keyword'] = df['Events'].apply(lambda x: x['keyword'])
        df['Lemma'] = df['Events'].apply(lambda x: x['lemma'])
        df['Keyword_Position'] = df['Events'].apply(lambda x: x['keyword_position'])
        
    if type == "biolord":
        df['Similarity'] = df['Events'].apply(lambda x: x['similarity'])
        df['Similarity'] = df['Similarity'].apply(lambda x: {k:v for (k,v) in x.items() if k!="Alert And Oriented"})
        for ET in event_types:
            df[f"{ET}_similarity"] = df['Similarity'].apply(lambda x:x[ET])
            
    return df
df_dictionary = prepare_df(pd.read_pickle(f"../exports/03_selected_reports_with_event_log_only_dictionary_v{version}/combined.pkl"),type="dictionary")
# df_biolord = prepare_df(pd.read_pickle(f"../exports/selected_reports_with_event_log_only_biolord_v{version}/combined.pkl"), type = "biolord")
df_dictionary.to_pickle(f"../exports/04_dictionary_biolord_features.pkl")





Loaded file len:  14483
After exploding file len:  322079


In [3]:
N1 = 10

from config import event_types
import os
os.makedirs("../exports/04_groundtruth/P-SET/Generated", exist_ok=True)
for ET in event_types:
    df_dictionary[f'Event_Name_{ET}'] = df_dictionary['Event_Name'].apply(lambda x: ET in x)
    row_id_to_doc_ET = df_dictionary.groupby('ROW_ID')[f'Event_Name_{ET}'].max().to_dict()
    df_dictionary[f'Event_Name_{ET}_doc'] = df_dictionary['ROW_ID'].apply(lambda x: row_id_to_doc_ET[x])
    people_ranked_by_ET = df_dictionary.groupby('SUBJECT_ID')[f'Event_Name_{ET}'].sum().sort_values(ascending=False).index.tolist()
    df_dictionary_topN1 = df_dictionary[df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N1])]
    df_dictionary_rest = df_dictionary[~df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N1])]
    Sentences_with_ET = pd.concat([df_dictionary_topN1[df_dictionary_topN1[f'Event_Name_{ET}']],df_dictionary_rest[df_dictionary_rest[f'Event_Name_{ET}']]])
    Sentences_with_ET['is_keyword_present'] = True
    # find N2 such that len(df_dictionary[~df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N1])]) <= len(Sentences_with_ET)
    N2 = N1
    while len(df_dictionary[(df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N2])) & ~df_dictionary[f'Event_Name_{ET}']]) >= len(Sentences_with_ET):
        N2 -= 1
    df_dictionary_topN2 = df_dictionary[df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N2])]
    Sentences_without_ET = df_dictionary_topN2[~df_dictionary_topN2[f'Event_Name_{ET}']]
    Sentences_without_ET['is_keyword_present'] = False
    Sentences = pd.concat([Sentences_with_ET, Sentences_without_ET]).reset_index(drop=True)
    Documents_with_ET = df_dictionary_topN1[df_dictionary_topN1[f'Event_Name_{ET}_doc']]
    Documents_with_ET['is_keyword_present'] = True
    Documents_without_ET = df_dictionary_topN1[~df_dictionary_topN1[f'Event_Name_{ET}_doc']]
    Documents_without_ET['is_keyword_present'] = False
    Documents = pd.concat([Documents_with_ET, Documents_without_ET]).reset_index(drop=True)
    Documents['Document'] = Documents['Document'].apply(lambda x: x.lower())
    print(f"For event type {ET}, N1={N1}, N2={N2}, Sentences_with_ET={len(Sentences_with_ET)}, Sentences_without_ET={len(Sentences_without_ET)}, Sentences={len(Sentences)},Documents_with_ET={Documents_with_ET.ROW_ID.nunique()}, Documents_without_ET={Documents_without_ET.ROW_ID.nunique()}, Documents={Documents.ROW_ID.nunique()}")
    Sentences.to_pickle(f"../exports/04_groundtruth/P-SET/Generated/{ET}_Sentences.pkl")
    Documents.to_pickle(f"../exports/04_groundtruth/P-SET/Generated/{ET}_Documents.pkl")

/tmp/ipykernel_58696/3638578464.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sentences_without_ET['is_keyword_present'] = False
/tmp/ipykernel_58696/3638578464.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_58696/3638578464.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

For event type Pain, N1=10, N2=10, Sentences_with_ET=9762, Sentences_without_ET=5216, Sentences=14978,Documents_with_ET=145, Documents_without_ET=80, Documents=225


/tmp/ipykernel_58696/3638578464.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sentences_without_ET['is_keyword_present'] = False
/tmp/ipykernel_58696/3638578464.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_58696/3638578464.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

For event type Sleep, N1=10, N2=5, Sentences_with_ET=2502, Sentences_without_ET=2366, Sentences=4868,Documents_with_ET=81, Documents_without_ET=121, Documents=202


/tmp/ipykernel_58696/3638578464.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sentences_without_ET['is_keyword_present'] = False
/tmp/ipykernel_58696/3638578464.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_58696/3638578464.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

For event type Excretion, N1=10, N2=7, Sentences_with_ET=5539, Sentences_without_ET=4979, Sentences=10518,Documents_with_ET=144, Documents_without_ET=173, Documents=317


/tmp/ipykernel_58696/3638578464.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sentences_without_ET['is_keyword_present'] = False
/tmp/ipykernel_58696/3638578464.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_58696/3638578464.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

For event type Eating, N1=10, N2=2, Sentences_with_ET=1829, Sentences_without_ET=1212, Sentences=3041,Documents_with_ET=96, Documents_without_ET=258, Documents=354
For event type Family, N1=10, N2=10, Sentences_with_ET=7766, Sentences_without_ET=6713, Sentences=14479,Documents_with_ET=170, Documents_without_ET=137, Documents=307


/tmp/ipykernel_58696/3638578464.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sentences_without_ET['is_keyword_present'] = False
/tmp/ipykernel_58696/3638578464.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_58696/3638578464.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [4]:
Documents.Document.iloc[0]

"ccu nsg admit note.\n19 year old female admitted to ccu status post vf arrest.\n\npmh:not significant.\n\nallergies:nkda.\n\nmeds:multiple over the counter diet supressents.\n\nhistory:?viral syndrome approx 2 weeks ago. taking diet surpressents- ?onset of use (per family planning [**state 2968**] vacation over holiday-?ing onset of diet surpressents). [**1-8**] am onset acute dyspnea w progression to cardiac arrest-vf. intubated & defib to st in field-transported to [**hospital1 2**]. aggressively rxed in ew. ct head/chest-neg for intracranial hemorrhage & pe. febrile-pan cultured & abx started. progressively deteriorated- requiring pressors & approx 4l fl-to card cath lad-clean c's, but elevated filling pressures-w 30's-rx w lasix & admitted to ccu for further management.\n   echo=sever global lv hk. lv function severly depressed. rv function depressed. 1+mr.\n\nsocial:bu student-2nd year. from ill. parents contact[**name (ni) **] & both present. has 2 other siblings in ill. non smo

In [ ]:
# N = 10
# from config import event_types
# import os
# os.makedirs("../exports/04_groundtruth/P-SET/Generated", exist_ok=True)
# for ET in event_types:
#     top_ET = df_dictionary_exploded[df_dictionary_exploded.Event_Name==ET][["SUBJECT_ID","Event_Name"]].value_counts().reset_index().iloc[:N]
#     selected_patient_sentences = df_dictionary[df_dictionary.SUBJECT_ID.isin(top_ET['SUBJECT_ID'].tolist())]
#     total_ET = top_ET[top_ET.Event_Name==ET]['count'].sum()
#     num_sentences = len(selected_patient_sentences.UID.unique())
#     num_reports = len(selected_patient_sentences.ROW_ID.unique())
#     num_patients = len(selected_patient_sentences.SUBJECT_ID.unique())
#     print(f"***{ET}***",
#           f"{ET} counts:{total_ET}\n",
#         # f"Event counts:\n{selected_patient_sentences.Event_Name.value_counts()}\n"
#         f"Unique Sentences: {num_sentences}\n",
#         f"Unique Reports: {num_reports}\n",
#         f"Unique Patients: {num_patients}\n"
#     )
#     selected_patient_sentences.to_pickle(f"../exports/04_groundtruth/P-SET/Generated/04_{ET}_{num_patients}_{num_reports}_{num_sentences}_{total_ET}.pkl")

***Pain*** Pain counts:422
 Unique Sentences: 5620
 Unique Reports: 225
 Unique Patients: 10

***Sleep*** Sleep counts:164
 Unique Sentences: 4783
 Unique Reports: 210
 Unique Patients: 10

***Excretion*** Excretion counts:229
 Unique Sentences: 7382
 Unique Reports: 317
 Unique Patients: 10

***Eating*** Eating counts:152
 Unique Sentences: 7267
 Unique Reports: 329
 Unique Patients: 10

***Family*** Family counts:536
 Unique Sentences: 7910
 Unique Reports: 328
 Unique Patients: 10



In [2]:
import pandas as pd
df = pd.read_pickle(f"../exports/04_dictionary_biolord_features.pkl")

In [3]:
ET = 'Sleep'
df_sentence_with_sleep = (df[df.Event_Name.apply(lambda x: ET in x)])
df_sentence_without_sleep = (df[df.Event_Name.apply(lambda x: ET not in x)])
top_10_patients = df_sentence_with_sleep.SUBJECT_ID.value_counts().head(10).index.tolist()
df_top_10_patient_sentences_with_sleep = df_sentence_with_sleep[df_sentence_with_sleep.SUBJECT_ID.isin(top_10_patients)]
df_top_10_patient_sentences_without_sleep = df_sentence_without_sleep[df_sentence_without_sleep.SUBJECT_ID.isin(top_10_patients)]
df_not_top_10_patient_sentences = df_sentence_with_sleep[~df_sentence_with_sleep.SUBJECT_ID.isin(top_10_patients)]
print("top 10 patient sentences:", len(df_top_10_patient_sentences_with_sleep))
print("top 10 patients sentences without sleep:", len(df_top_10_patient_sentences_without_sleep))
print("other patient sentences:", len(df_not_top_10_patient_sentences))

top 10 patient sentences: 147
top 10 patients sentences without sleep: 4612
other patient sentences: 2355


In [4]:
df_top_10_patient_sentences_without_sleep.SUBJECT_ID.value_counts()

SUBJECT_ID
15883    749
23082    604
16459    541
22673    522
4874     489
25328    473
14995    453
3748     361
32291    271
7042     149
Name: count, dtype: int64

In [19]:
docid_isevent = df[df.SUBJECT_ID.isin(top_10_patients)].groupby(['ROW_ID','SUBJECT_ID'])['Event_Name'].agg(lambda x: any([ET in j for j in x])).reset_index()
docid_isevent[docid_isevent.Event_Name==True]['SUBJECT_ID'].value_counts()
res = docid_isevent.groupby(["Event_Name", "SUBJECT_ID"]).size().unstack(fill_value=0)
res, res.sum(axis=1)

(SUBJECT_ID  3748   4874   7042   14995  15883  16459  22673  23082  25328  \
 Event_Name                                                                  
 False           7      7      6      4     28     18     12     24     10   
 True            7      9      5     12      8      5     10      8      8   
 
 SUBJECT_ID  32291  
 Event_Name         
 False           5  
 True            9  ,
 Event_Name
 False    121
 True      81
 dtype: int64)

In [20]:
sentid_isevent = df[df.SUBJECT_ID.isin(top_10_patients)].groupby(['UID','SUBJECT_ID'])['Event_Name'].agg(lambda x: any([ET in j for j in x])).reset_index()
sentid_isevent[sentid_isevent.Event_Name==True]['SUBJECT_ID'].value_counts()
res = sentid_isevent.groupby(["Event_Name", "SUBJECT_ID"]).size().unstack(fill_value=0)
res, res.sum(axis=1)

(SUBJECT_ID  3748   4874   7042   14995  15883  16459  22673  23082  25328  \
 Event_Name                                                                  
 False         361    489    149    453    749    541    522    604    473   
 True           15     17     12     24     13     14     15     12     12   
 
 SUBJECT_ID  32291  
 Event_Name         
 False         271  
 True           13  ,
 Event_Name
 False    4612
 True      147
 dtype: int64)

In [50]:
for N2 in range(7):
    res = df_sentence_without_sleep[df_sentence_without_sleep.SUBJECT_ID.isin(top_10_patients[:N2])]
    print(N2,len(res))
    
2366 + 2502, 121 + 81

0 0
1 453
2 942
3 1464
4 1825
5 2366
6 2637


(4868, 202)

In [49]:
df[df.SUBJECT_ID.isin(top_10_patients)].ROW_ID.nunique()

202

In [35]:
2355 + 147

2502

df.LOS